<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Gradient Descent in Scikit-Learn

_Authors: Kiefer Katovich (SF)_

---

Until now we've been using specific scikit-learn model classes such as `LinearRegression` and `LogisticRegression` to perform regression and classification. While these methods work well on smaller data sets with relatively few features, the process slows down to a crawl once you start getting into "medium data." Plus, these methods take up so much memory that fitting them becomes mind-numbingly slow — especially on a laptop.

Luckily, scikit-learn comes with stochastic gradient descent solvers for regression and classification:
- `SGDRegressor`
- `SGDClassifier`

Because of its ability to minimize the loss function iteratively on smaller portions of the data, the SGD solvers avoid the intense slowdown other models suffer on large data sets.

> **Note:** The gradient descent solvers are flexible and can fit a variety of different model types not covered here. We highly recommend reading their documentation in detail.

---

### San Francisco Assessor Data

This lab uses data on housing prices in San Francisco from the S.F. Assessor's Office — the set is already cleaned up.

You can see that the set has 250,000 rows. When expanding this with dummy-coded categorical columns, it can become quite large. Be careful that you don't exceed the memory on your computer.


In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import scipy.stats as stats

import patsy

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

### 1. Load the data.

Examine the columns.

In [2]:
prop = pd.read_csv('../datasets/assessor_sample.csv')

In [3]:
prop.shape

(250000, 17)

In [4]:
prop.dtypes

baths               int64
beds                int64
lot_depth         float64
basement_area     float64
front_ft          float64
owner_pct         float64
rooms               int64
property_class     object
neighborhood       object
tax_rate          float64
volume              int64
sqft                int64
stories             int64
year_recorded       int64
year_built          int64
zone               object
value             float64
dtype: object

In [5]:
prop.isnull().sum()

baths             0
beds              0
lot_depth         0
basement_area     0
front_ft          0
owner_pct         0
rooms             0
property_class    0
neighborhood      0
tax_rate          0
volume            0
sqft              0
stories           0
year_recorded     0
year_built        0
zone              0
value             0
dtype: int64

In [6]:
prop.property_class.unique()

array(['D', 'Z', 'DBM', 'LZ', 'TH', 'ZBM'], dtype=object)

In [7]:
prop.neighborhood.unique()

array(['01E', '10G', '10C', '02C', '02A', '04S', '08F', '10H', '03A',
       '02F', '10B', '04T', '04J', '04M', '08E', '10F', '10E', '03G',
       '03H', '06D', '09E', '02B', '02D', '10D', '01C', '07B', '02E',
       '04C', '04H', '10A', '05A', '04F', '04E', '10J', '09A', '03B',
       '07C', '01A', '01G', '09G', '02G', '05C', '05F', '03J', '05K',
       '03E', '06A', '05D', '08C', '04G', '04R', '06B', '01D', '07A',
       '09C', '09F', '10K', '01F', '05G', '03C', '09H', '04B', '08G',
       '04N', '05B', '08A', '08H', '04P', '03F', '07D', '06E', '01B',
       '05M', '05E', '06C', '04D', '05H', '08B', '05J', '09D', '04A',
       '04K', '08D', '06F', '03D', '09B', '047', '08I'], dtype=object)

### 2) Sample down the data.

Even though this is already only a sample of the full assessor data set, you should sample the data down further for the sake of speed and your computer's memory.

Use the `.sample()` function for Pandas DataFrames to subset this down to < 25,000 rows. 

Sampling down large data sets is a common procedure. Finding the optimal parameters with larger subsets of the data may change the hyperparameters and results and will get you closer to the best coefficients — but the returns become marginal at a certain point.

In [8]:
prop_samp = prop.sample(n=25000)

### 3) Regression with stochastic gradient descent.

Below are x, y data predicting value (housing price) from the remaining variables. There are ~75,000 rows with 170 columns.


The `SGDRegressor` is general and flexible and can be customized with a variety of keyword arguments.

**Arguments**
- `loss`: `['squared_loss','huber', ...]`
    - The `'squared_loss'` corresponds to solving a regression with the least squares loss. This is what you'll probably use, but there are other options. Huber loss is a "robust" regression loss.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression you'd like to solve. The l1 and l2 are the lasso and ridge, while the elastic net is the combination of both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. It's the same as in the lasso and ridge.
- `l1_ratio`
    - The mix of the lasso and ridge penalties when elastic net is chosen as the penalty.
- `n_iter`
    - The number of training epochs over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to five).

`SGDRegressor` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It's up to you how you want to define the model. You should:**

1) Choose a target to estimate (this should be continuous).
    - Select predictors to use.
    - Standardize your predictor matrix.
    - Build a stochastic gradient descent solver to fit your model. You'll likely want to perform some kind of grid search to find the optimal parameters for your model.
    - Describe the model selected through grid search and compare the performance to the baseline.
    - Examine and interpret the coefficients.

In [10]:
f = 'value ~ '+' + '.join([c for c in prop_samp.columns if not c == 'value'])+' -1'
print(f)
y, X = patsy.dmatrices(f, data=prop_samp, return_type='dataframe')
y = y.values.ravel()

print(y.shape, X.shape)

value ~ baths + beds + lot_depth + basement_area + front_ft + owner_pct + rooms + property_class + neighborhood + tax_rate + volume + sqft + stories + year_recorded + year_built + zone -1
(25000,) (25000, 163)


In [11]:
# I am predicting the value of the house from the other variables.

In [12]:
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [14]:
# Standardize the predictors
ss = StandardScaler()
Xs = ss.fit_transform(X)
print(Xs.shape)

(25000, 163)


In [15]:
# Set up the gridsearch parameters:
sgd_params = {
    'loss':['squared_loss','huber'],
    'penalty':['l1','l2'],
    'alpha':np.logspace(-5,1,25)
}

sgd_reg = SGDRegressor()
sgd_reg_gs = GridSearchCV(sgd_reg, sgd_params, cv=5, verbose=False)

In [16]:
# SGD is pretty fast compared to other sklearn solvers - but can still
# take a good long while depending on the gridsearch and the size of
# the dataset.
sgd_reg_gs.fit(Xs, y)

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

...

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWar

GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'loss': ['squared_loss', 'huber'], 'penalty': ['l1', 'l2'], 'alpha': array([1.00000e-05, 1.77828e-05, 3.16228e-05, 5.62341e-05, 1.00000e-04,
       1.77828e-04, 3.16228e-04, 5.62341e-04, 1.00000e-03, 1.77828e-03,
       3.16228e-03, 5.62341e-03, 1.00000e-02, 1.77828e-02, 3.16228e-02,
       5.62341e-02, 1.00000e-01, 1.77828e-01, 3.16228e-01, 5.62341e-01,
       1.00000e+00, 1.77828e+00, 3.16228e+00, 5.62341e+00, 1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=False)

In [17]:
print(sgd_reg_gs.best_params_)
print(sgd_reg_gs.best_score_)
sgd_reg = sgd_reg_gs.best_estimator_

{'alpha': 0.5623413251903491, 'loss': 'squared_loss', 'penalty': 'l2'}
0.22857567873994103


In [18]:
# We can see that the gradient descent got a .22 R2 using the ridge and squared loss.

In [19]:
value_coefs = pd.DataFrame({'coef':sgd_reg.coef_,
                            'mag':np.abs(sgd_reg.coef_),
                            'pred':X.columns})
value_coefs.sort_values('mag', ascending=False, inplace=True)
value_coefs.iloc[0:10, :]

,coef,mag,pred
159,38948.102832,38948.102832,sqft
151,38885.816476,38885.816476,beds
72,32406.225855,32406.225855,neighborhood[T.08G]
150,27698.787323,27698.787323,baths
47,27273.276864,27273.276864,neighborhood[T.05C]
70,27187.947733,27187.947733,neighborhood[T.08E]
121,-25569.617068,25569.617068,zone[T.RH1RM1]
140,24673.271359,24673.271359,zone[T.RMZ]
63,22309.171231,22309.171231,neighborhood[T.07B]
155,-22182.980134,22182.980134,owner_pct


### 4) Classification with stochastic gradient descent.

The `SGDClassifier` is very similar to the `SGDRegressor`. The main difference is that the loss functions are changed to regression loss functions.

**Arguments**
- `loss`: `['log', ...]`
    - The `'log'` loss corresponds to solving a logistic regression classifier. This is what you'll probably use, but there are other options.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression you'd like to solve. The l1 and l2 are the lasso and ridge, while the elastic net is the combination of both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. It's the same as in the lasso and ridge.
- `l1_ratio`
    - The mix of the lasso and ridge penalties when elastic net is chosen as the penalty.
- `n_iter`
    - The number of training epochs over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to five).

Like `SGDRegressor`, `SGDClassifier` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It's up to you how you want to define the model. You should:**

1) Choose a target to classify (you may need to engineer one from existing variables).
    - Calculate the baseline accuracy.
    - Select predictors to use.
    - Standardize your predictor matrix.
    - Build a stochastic gradient descent solver to fit your model. You'll likely want to perform some kind of grid search to find the optimal parameters for your model.
    - Describe the model selected through grid search and compare the performance to baseline.
    - Examine and interpret the coefficients.

In [20]:
prop_samp.columns

Index(['baths', 'beds', 'lot_depth', 'basement_area', 'front_ft', 'owner_pct',
       'rooms', 'property_class', 'neighborhood', 'tax_rate', 'volume', 'sqft',
       'stories', 'year_recorded', 'year_built', 'zone', 'value'],
      dtype='object')

In [21]:
prop_samp.year_built.value_counts()

1941    967
1940    906
1925    900
1924    768
1926    738
1927    713
1923    649
1939    637
1947    579
1948    573
1908    503
1928    493
1946    466
1938    464
1950    445
1906    443
1922    434
1907    430
1931    412
1951    404
1936    356
1944    352
1929    345
1910    334
1949    323
1930    317
1912    311
1942    307
1937    298
1916    268
       ... 
1995    106
1993    105
1933    105
1997    101
1998    100
1965     96
1994     86
1977     79
1985     73
1966     69
1973     65
1902     63
1903     56
1970     52
1968     52
1918     49
1974     49
1967     47
1934     46
1976     43
1969     28
2000     27
1971     27
1999     22
1901     20
2001      9
2003      4
2002      1
2004      1
2005      1
Name: year_built, Length: 105, dtype: int64

In [22]:
# Lets see if we can predict if a house was built past 1980
prop_samp['built_past1980'] = prop_samp.year_built.map(lambda x: 1 if x >= 1980 else 0)

In [23]:
# Make the target and calculate the baseline:
y = prop_samp.built_past1980.values
print(1. - np.mean(y))

0.89432


In [24]:
f = '''
~ baths + beds + lot_depth + basement_area + front_ft + owner_pct +
rooms + property_class + neighborhood + tax_rate + volume + sqft + stories +
zone + value -1
'''

In [25]:
X = patsy.dmatrix(f, data=prop_samp, return_type='dataframe')

Xs = ss.fit_transform(X)
print(y.shape, Xs.shape)

(25000,) (25000, 162)


In [26]:
sgd_cls_params = {
    'loss':['log'],
    'penalty':['l1','l2'],
    'alpha':np.logspace(-5,2,50)
}

sgd_cls = SGDClassifier()
sgd_cls_gs = GridSearchCV(sgd_cls, sgd_cls_params, cv=5, verbose=1)

In [27]:
sgd_cls_gs.fit(Xs, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

...

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

GridSearchCV(cv=5, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'loss': ['log'], 'penalty': ['l1', 'l2'], 'alpha': array([1.00000e-05, 1.38950e-05, 1.93070e-05, 2.68270e-05, 3.72759e-05,
       5.17947e-05, 7.19686e-05, 1.00000e-04, 1.38950e-04, 1.93070e-04,
       2.68270e-04, 3.72759e-04, 5.17947e-04, 7.19686e-04, 1.00000e-03,
       1.38950e-03, 1... 1.38950e+01, 1.93070e+01,
       2.68270e+01, 3.72759e+01, 5.17947e+01, 7.19686e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [28]:
print(sgd_cls_gs.best_params_)
print(sgd_cls_gs.best_score_)
sgd_cls = sgd_cls_gs.best_estimator_

{'alpha': 0.00026826957952797245, 'loss': 'log', 'penalty': 'l1'}
0.96488
